# Hello from JupyterLite + Voici
This notebook is packaged to run **entirely in your browser**.

Use the widget below to change the sine wave frequency. It works in **JupyterLab** and in **Voici**.


In [ ]:
import numpy as np
import ipywidgets as W
import matplotlib.pyplot as plt

%matplotlib inline

@W.interact(freq=(1, 12))
def plot(freq=5):
    x = np.linspace(0, 2*np.pi, 400)
    y = np.sin(freq * x)
    fig, ax = plt.subplots()
    ax.plot(x, y)
    ax.set_title(f"Sine wave — freq={freq}")
    ax.set_xlabel("x")
    ax.set_ylabel("sin(freq*x)")
    plt.show()
